In [1]:
# test Gurobi installation
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import requests  # for loading the example source code
import openai

# import flaml and autogen
from flaml import autogen
from flaml.autogen.agentchat import Agent, UserProxyAgent
from optiguide.optiguide import OptiGuideAgent

In [2]:
config_list = [
    {
        "api_type": "open_ai",
        "api_base": "http://localhost:1234/v1",
        "api_key": "NULL"
    }
]

llm_config = {
    "config_list": config_list,
    "seed": 47,
    "temperature": 0.1,
    "max_tokens": -1,
    "request_timeout": 6000
}

In [3]:
# Get the source code of our first example
code_url = "https://raw.githubusercontent.com/sayan-IIT/SchedulingGenAI/main/benchmark/application/schedule.py"
response  = requests.get(code_url)


# Check if the request was successful
if response.status_code == 200:
    # Get the text content from the response
    code = response.text
else:
    raise RuntimeError("Failed to retrieve the file.")
# code = open(code_url, "r").read() # for local files


# show the first head and tail of the source code
print("\n".join(code.split("\n")[:10]))
print(".\n" * 3)
print("\n".join(code.split("\n")[-10:]))

import time
from gurobipy import GRB, Model

# Example data

capacity_in_technician = {'Adam': 150, 'John': 50, 'Dave': 100}

Technician_Cost = {
    ('Adam', 'Internal'): 5,
    ('Adam', 'External'): 4,
.
.
.

# Solve
m.update()
model.optimize()

print(time.ctime())
if m.status == GRB.OPTIMAL:
    print(f'Optimal cost: {m.objVal}')
else:
    print("Not solved to optimality. Optimization status:", m.status)



In [4]:
# In-context learning examples.
example_qa = """
----------
Question: Why is it not recommended to use just one technician for internal?
Answer Code:
```python
z = m.addVars(technicians, vtype=GRB.BINARY, name="z")
m.addConstr(sum(z[s] for s in technicians) <= 1, "_")
for s in technicians:
    m.addConstr(x[s,'internal'] <= capacity_in_technician[s] * z[s], "_")
```

----------
Question: What if there's a 13% jump in the demand for maintenance at Northcote?
Answer Code:
```python
maintenance_work["Northcote"] = maintenance_work["Northcote"] * (1 + 13/100)
```
"""

In [6]:
%%capture
agent = OptiGuideAgent(
    name="GenIE",
    source_code=code,
    debug_times=1,
    example_qa="",
    llm_config=llm_config
)

user = UserProxyAgent(
    "user", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

In [9]:
user.initiate_chat(agent, message="What if we have no technicians which are external?")

user (to GenIE):

What if we have no technicians which are external?

--------------------------------------------------------------------------------
GenIE (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to GenIE):

 Here is an example of how you could modify the code to handle the case where there are no technicians that are external:
```python
import time
from gurobipy import GRB, Model

# Example data

capacity_in_technician = {'Adam': 150, 'John': 50, 'Dave': 100}

Technician_Cost = {
    ('Adam', 'Internal'): 5,
    ('Adam', 'External'): 20,
    ('John', 'Internal'): 10,
    ('John', 'External'): 30,
    ('Dave', 'Internal'): 15,
    ('Dave', 'External'): 40
}

Travel_Cost = {
    ('Adam', 'Internal'): {'Alice': 20, 'Bob': 30},
    ('Adam', 'External'): {'Alice': 80, 'Bob': 100},
    ('John', 'Internal'): {'Alice': 40, 'Bob': 50},
    ('John', 'External'): {'Alice': 120, 'Bob': 160},
    ('Dave', 'Internal'): 

In [8]:
user.initiate_chat(agent, message="What is the impact of Adam being able to work only half the time at present?")

user (to GenIE):

What is the impact of Adam being able to work only half the time at present?

--------------------------------------------------------------------------------
GenIE (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to GenIE):

 Here's an example code snippet that you can add after the line "# OPTIGUIDE *** CODE GOES HERE":
```python
# If Adam is able to work only half the time at present, we need to adjust his capacity accordingly.
capacity_in_technician['Adam'] = 75
```
This code snippet reduces the capacity of Adam from 150 to 75. You can adjust this value based on the actual capacity of Adam.

--------------------------------------------------------------------------------
GenIE (to safeguard):


--- Code ---
# If Adam is able to work only half the time at present, we need to adjust his capacity accordingly.
capacity_in_technician['Adam'] = 75

--- One-Word Answer: SAFE or DANGER ---


-----------